# CNN MNIST Classifier
Objective: build a convolutional neural network to classify MNIST handwritten digits dataset<br><br>
Based in Convolutional Neural Network Example
- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/ 

## MNIST Dataset Overview
The dataset contains 60,000 examples for training and 10,000 examples for testing. <br>The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1.<br> For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

In [1]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf

# import mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Hiperparameters

In [2]:
# training parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# network parameters
num_input = 784
num_classes = 10
dropout = 0.75

# tensorflow graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

### Convolution and polling functions

In [3]:
# two functions: convolution and pooling
def conv2d(x, W, b, strides = 1):
  x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = "SAME")
  x = tf.nn.bias_add(x, b)
  return tf.nn.relu(x)

def maxpool2d(x, k = 1):
  return tf.nn.max_pool(x, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'SAME')

### Convolution Neuro Network model

In [4]:
def conv_net(x, weights, biases, dropout):
  x = tf.reshape(x, shape = [-1, 28, 28, 1])
  # layer 1 - convolution
  conv1 = conv2d(x, weights['wc1'], biases['bc1'])
  conv1 = maxpool2d(conv1, k = 2)
  # layer 2 - convolution
  conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
  conv2 = maxpool2d(conv2, k = 2)
  # layer 3 - fully connected
  fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
  fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
  fc1 = tf.nn.relu(fc1)
  fc1 = tf.nn.dropout(fc1, dropout)
  # output
  out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
  return out

In [5]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), 
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])), 
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])), 
    'out': tf.Variable(tf.random_normal([num_classes])), 
}

# model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

# model evaluation
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variables initializing
init = tf.global_variables_initializer()

### Training the model

In [6]:
import time
start_time = time.time()

with tf.Session() as sess:
  sess.run(init)
  
  for step in range(1, num_steps + 1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    # backpropagation
    sess.run(train_op, feed_dict = {X: batch_x, Y: batch_y, keep_prob: dropout})
    # step display
    if step % display_step == 0 or step == 1:
      # loss and accuracy
      loss, acc = sess.run([loss_op, accuracy], feed_dict = {X: batch_x, 
                                                            Y: batch_y, 
                                                            keep_prob: 1.0})
      print('Step ' + str(step) + \
           ', Minibatch Loss = ' + "{:.4f}".format(loss) + \
           ', Training Accuracy = ' + "{:.3f}".format(acc))
  print("Optization finished!")
 
  
  # accuracy for 255 mnist test images
  print("Testing Accuracy: ", \
       sess.run(accuracy, feed_dict = {X: mnist.test.images[:256], 
                                      Y: mnist.test.labels[:256], 
                                      keep_prob: 1.0}))
  
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))


Step 1, Minibatch Loss = 83072.2344, Training Accuracy = 0.078
Step 10, Minibatch Loss = 27916.9473, Training Accuracy = 0.234
Step 20, Minibatch Loss = 12005.0391, Training Accuracy = 0.453
Step 30, Minibatch Loss = 6906.5488, Training Accuracy = 0.695
Step 40, Minibatch Loss = 7788.3496, Training Accuracy = 0.711
Step 50, Minibatch Loss = 5223.2505, Training Accuracy = 0.781
Step 60, Minibatch Loss = 4330.4404, Training Accuracy = 0.828
Step 70, Minibatch Loss = 2746.0620, Training Accuracy = 0.844
Step 80, Minibatch Loss = 4330.0601, Training Accuracy = 0.805
Step 90, Minibatch Loss = 3387.0996, Training Accuracy = 0.867
Step 100, Minibatch Loss = 2885.4822, Training Accuracy = 0.828
Step 110, Minibatch Loss = 1962.0945, Training Accuracy = 0.883
Step 120, Minibatch Loss = 1708.4531, Training Accuracy = 0.930
Step 130, Minibatch Loss = 3147.8540, Training Accuracy = 0.867
Step 140, Minibatch Loss = 3096.9431, Training Accuracy = 0.891
Step 150, Minibatch Loss = 2137.5874, Training A

'00:05:38'